# This notebook handles the data treatment for the dataset found at:
https://github.com/LumosBio/MolData/blob/main/Data/all_molecular_data.zip


# Data ingestion


In [1]:
# Basic imports
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
# Build SparkSession
spark = SparkSession.builder.appName("JoinWindowingSQL").getOrCreate()

In [7]:
data_dir ='../Datasets/'
mol_data = data_dir + 'all_molecular_data.csv'

In [4]:
! head $mol_data

^C


In [8]:
df_mol = spark.read.csv(mol_data, header=True, sep=',', inferSchema=True)

In [10]:
df_mol.show(10)

+--------------------+-----------+-------------+-------------+-------------+-------------+-------------+------------+------------+----------------+---------------+---------------+----------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+-------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+-------------+-------------+---------------+---------------+-------------+-------------+-------------+-------------+-------------+---